In [152]:
import time
import json
import urllib3
import requests
from datetime import datetime, timezone, timedelta, date
from opensearchpy import OpenSearch
from opensearch_helper import OSWriter  
import calendar
import logging
from collections import defaultdict

In [153]:
year = datetime.utcnow().year
fromTime = f"{year}-03-31T18:29:00Z"
toTime = (datetime.utcnow() - timedelta(days=1)).strftime("%Y-%m-%dT18:31:00Z")
profile = "1-0:99.2.0*255"  # daily_profile
ALLOWED_REGS = {"1-0:1.8.0*255", "1-0:9.8.0*255"}  # kwh_register, kvah_register

In [154]:
def get_devices():
    notOver = True
    while notOver:
        try:
            r = requests.get(url1, verify=False, auth=(muser, msecret))
            r.raise_for_status()
            notOver = False
        except requests.exceptions.HTTPError as errh:
            continue
        except requests.exceptions.ConnectionError as errc:
            continue
        except requests.exceptions.Timeout as errt:
            continue
        except requests.exceptions.RequestException as err:
            continue
    devList = json.loads(r.content.decode('utf-8'))
    return devList

deviceMasterList = get_devices()
print(f"Total devices fetched: %d", len(deviceMasterList))

devicesFiltered = []
for item in deviceMasterList:
    if item.get("inventoryState") == "installed":
        groupName = " ".join(
            w for w in item.get("groupName", "").split()
            if w.lower() not in {"prepaid", "postpaid"}
        )
        devicesFiltered.append({"deviceId": item['id'], "groupName": groupName})
print(f"Total installed devices: %d", len(devicesFiltered))

Total devices fetched: %d 11247
Total installed devices: %d 10573


In [ ]:
#daily_profile: 1-0:99.2.0*255
#kwh_register:1-0:1.8.0*255
#kvah_register:1-0:9.8.0*255

In [156]:
def get_daily_profile(fromTime, toTime, profile):
    batchSize = 2000
    count = 1
    postData = []
    batchDeviceList = []
    batchGroupList = []
    deviceList = []
    groupList = []
    for item in devicesFiltered:
        deviceList.append(item['deviceId'])
        groupList.append(item["groupName"])
    # Accumulator for all returned rows across batches
    data_dictionary = []

    # url2 already set above
    for idx, deviceId in enumerate(deviceList):
        postData.append({"device": deviceId, "profile": profile, "from": fromTime, "to": toTime})
 #       print(postData)
#        break
        batchDeviceList.append(deviceId)
        batchGroupList.append(groupList[idx])

        if (len(postData) == batchSize) or (count == len(devicesFiltered)):
            notOver = True
            while notOver:
                try:
                    r = requests.post(url2, json=postData, verify=False, auth=(muser, msecret))
                    r.raise_for_status()
                    notOver = False
                except requests.exceptions.HTTPError as errh:
                    print("HTTP Error: %s", errh)
                    continue
                except requests.exceptions.ConnectionError as errc:
                    print("Connection Error: %s", errc)
                    continue
                except requests.exceptions.Timeout as errt:
                    print("Timeout Error: %s", errt)
                    continue
                except requests.exceptions.RequestException as err:
                    print("Other Error: %s", err)
                    continue

            returnJSON = json.loads(r.content.decode('utf-8'))

            # Map returned entries back to the batch device/group lists by position
            j = 0
            for entry in returnJSON:
                if entry.get("success"):
                    valueList = entry.get("value", [])
                    for value in valueList:
                        for registerValue in value.get("meteredValues", []):
                            reg_id = registerValue.get("registerId")
                            if not reg_id or reg_id not in ALLOWED_REGS:
                                continue
                            data_dictionary.append({
                                "deviceId": batchDeviceList[j],
                                "groupName": batchGroupList[j],
                                "registerId": registerValue.get("registerId"),  # <-- fixed name
                                "unit": registerValue.get("unit"),
                                "value": registerValue.get("value"),
                                "measuredAt": registerValue.get("measuredAt")
                            })
                j += 1

            # Reset for next batch
            postData = []
            batchDeviceList = []
            batchGroupList = []
            print("Completed batch %d", count)

        count += 1
    return data_dictionary


In [157]:
dp_data=get_daily_profile(fromTime,toTime,profile)

Completed batch %d 2000
Completed batch %d 4000
Completed batch %d 6000
Completed batch %d 8000
Completed batch %d 10000
Completed batch %d 10573


In [123]:
dp_data_kwh = []
dp_data_kvah = []
for item in dp_data:
    if item['registerId'] == "1-0:1.8.0*255":
        dp_data_kwh.append(item)
    elif item['registerId'] == "1-0:9.8.0*255":
        dp_data_kvah.append(item)
print(len(dp_data_kwh), len(dp_data_kvah))
print(dp_data_kvah[:1])

1389299 1389299
[{'deviceId': 'ISKIE070642', 'groupName': 'Durgapur Steel TPS Domestic', 'registerId': '1-0:9.8.0*255', 'unit': 'kVAh', 'value': 101.163, 'measuredAt': '2025-03-31T18:30:00Z'}]


In [158]:
# Step 1: Split dp_data into kWh and kVAh
dp_data_kwh = [d for d in dp_data if d['registerId'] == "1-0:1.8.0*255"]
dp_data_kvah = [d for d in dp_data if d['registerId'] == "1-0:9.8.0*255"]

print(len(dp_data_kwh), len(dp_data_kvah))
print(dp_data_kvah[:1])



1398076 1398076
[{'deviceId': 'ISKIE070642', 'groupName': 'Durgapur Steel TPS Domestic', 'registerId': '1-0:9.8.0*255', 'unit': 'kVAh', 'value': 101.163, 'measuredAt': '2025-03-31T18:30:00Z'}]


In [160]:
def get_consumption_per_device(devices, data):
    grouped = defaultdict(list)
    for entry in data:
        grouped[entry['deviceId']].append(entry)

    result = []
    now = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ")

    for dev in devices:
        dev_id = dev['deviceId']
        if dev_id in grouped:
            entries = grouped[dev_id]
            first, last = entries[0], entries[-1]
            diff = last['value'] - first['value']

            result.append({
                "deviceId": dev_id,
                "groupName": dev["groupName"],
                "registerId": first["registerId"],
                "unit": first["unit"],
                "value": diff,
                "measuredAt": now
            })
    return result


# Step 3: Apply helper for both register types
result_kwh = get_consumption_per_device(devicesFiltered, dp_data_kwh)
result_kvah = get_consumption_per_device(devicesFiltered, dp_data_kvah)

In [167]:
merged_result = result_kwh + result_kvah
merged_result.sort(key=lambda x: (x['deviceId'], x['registerId']))
for item in merged_result[:5]:
    print(item)

{'deviceId': 'EHLTNP36007', 'groupName': 'ROOT', 'registerId': '1-0:1.8.0*255', 'unit': 'kWh', 'value': 5499.788005000002, 'measuredAt': '2025-09-15T05:20:11Z'}
{'deviceId': 'EHLTNP36007', 'groupName': 'ROOT', 'registerId': '1-0:9.8.0*255', 'unit': 'kVAh', 'value': 6770.486539999998, 'measuredAt': '2025-09-15T05:20:12Z'}
{'deviceId': 'ISKER00001', 'groupName': 'Mejia TPS Domestic', 'registerId': '1-0:1.8.0*255', 'unit': 'kWh', 'value': 596.2199999999999, 'measuredAt': '2025-09-15T05:20:11Z'}
{'deviceId': 'ISKER00001', 'groupName': 'Mejia TPS Domestic', 'registerId': '1-0:9.8.0*255', 'unit': 'kVAh', 'value': 638.7699999999999, 'measuredAt': '2025-09-15T05:20:12Z'}
{'deviceId': 'ISKER00002', 'groupName': 'Mejia TPS Domestic', 'registerId': '1-0:1.8.0*255', 'unit': 'kWh', 'value': 1013.901, 'measuredAt': '2025-09-15T05:20:11Z'}


In [ ]:
for item in 